<a href="https://colab.research.google.com/github/Lin8823/Lin/blob/main/%E4%B8%80%E9%A8%B0%E8%B3%87%E8%A8%8A/land_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 54.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=0890124fb2e362e36a8974c7c65765549f56f07a61d346ac6344d16d9f6437ab
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from pyspark.context import SparkContext,SparkConf
from pyspark.sql.session import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import HiveContext
from pyspark.sql.functions import *
from pyspark.sql.types import StringType
from pyspark.sql.functions import concat

spark = SparkSession.\
            builder.\
            getOrCreate()


In [6]:
path = ['/content/A_lvr_land_A.csv', '/content/B_lvr_land_A.csv', '/content/E_lvr_land_A.csv', '/content/F_lvr_land_A.csv', '/content/H_lvr_land_A.csv']
df = spark.read.csv(path, header=True)

chinese_dict ={'零':0, '一':1, '二':2, '兩':2, '三':3, '四':4, '五':5, '六':6, '七':7, '八':8, '九':9, '十':10, 
                           '百':100, '千':1000, '萬':10000, '億':100000000}
def transform(uchars_chinese):
  total = 0
  r = 1              #表示單位：個十百千...
  try:
    for i in range(len(uchars_chinese) - 2, -1, -1): #排除資料中的"層"這個字後，從個位數開始進行數字轉換
      val = chinese_dict.get(uchars_chinese[i])
      if val >= 10 and i == 0:  #for 十開頭的，十三、十四、十*
        if val > r: #若大於現在的單位
          r = val #指派給r，進行下一輪單位比較或運算
          total = total + val
        else:
          r = r * val
      elif val >= 10:
        if val > r:
          r = val
        else:
          r = r * val
      else:
        total = total + r * val
  except TypeError:
    pass

  return total
transformUDF = udf(lambda z: transform(z)) 
df = df.withColumn("總樓層數_fix", transformUDF(df.總樓層數)) #使用自訂義函數轉換數字
df=df.filter((df.主要用途=='住家用') & (df.建物型態.rlike("住宅大樓.") & (df.總樓層數_fix >=13))).sort(desc('交易年月日')) #【主要用途】為【住家用】、【建物型態】為【住宅大樓】、【總樓層數】需【大於等於十三層】
df.show()

+--------+--------------------+----------------------------------------+----------------------+----------------+------------------+------------------+----------+---------------+--------+--------+--------------------------+--------+--------------+------------+----------------------+---------------+---------------+---------------+-----------------+------------+---------+--------------+--------+------------------------+----------+-------------------------------------+-------------------+----------+------------+--------+----+--------+------------+
|鄉鎮市區|            交易標的|                        土地位置建物門牌|土地移轉總面積平方公尺|都市土地使用分區|非都市土地使用分區|非都市土地使用編定|交易年月日|     交易筆棟數|移轉層次|總樓層數|                  建物型態|主要用途|      主要建材|建築完成年月|建物移轉總面積平方公尺|建物現況格局-房|建物現況格局-廳|建物現況格局-衛|建物現況格局-隔間|有無管理組織|   總價元|單價元平方公尺|車位類別|車位移轉總面積(平方公尺)|車位總價元|                                 備註|               編號|主建物面積|附屬建物面積|陽台面積|電梯|移轉編號|總樓層數_fix|
+--------+--------------------+----------------------------------------+----------------------

In [5]:
df.write.mode('overwrite').json('/content/result')